# AES
![flow](https://mjtoolbox.files.wordpress.com/2013/01/aes2.jpg)

## Something we should know
* Rijndael's finite field
    * Galois field $2^8$
    * operator
        1. add 
            - just xor
        2. multiply 2
            1. check msb for overflow
            2. shift left 1 for multiplication by 2
            3. if overflow happened, xor `0x11B` into GF
                - mod an irreducible reducing polynomia in GF($2^8$)
                - in char-type, we only use `0x1B` instead of `0x11B`
        3. multiply 3
            - multiply 2 and add 1
        4. mod
            - use xor instead of subtraction
            - an irreducible reducing polynomial
                * ${x^8} + {x^4} + {x^3} + x + 1$ in GF($2^8$)
* Extended Euclidean algorithm
    * use it to get modular multiplicative inverses in GF($2^8$)
* Linear algebra
    * linear transform a column into another space
    * it is invertible
    
* MixColumn
    * Transform
    $$
     \left[
     \begin{matrix}
       2 & 3 & 1 & 1 \\
       1 & 2 & 3 & 1 \\
       1 & 1 & 2 & 3 \\
       3 & 1 & 1 & 2
      \end{matrix}
      \right]
    $$
    * Inverse
    $$
     \left[
     \begin{matrix}
       14 & 11 & 13 & 9 \\
       9 & 14 & 11 & 13 \\
       13 & 9 & 14 & 11 \\
       11 & 13 & 9 & 14
      \end{matrix}
      \right]
    $$
    * you can use `Galois Multiplication lookup tables` to speedup the Calculation

In [1]:
from math import floor
#import numpy as np
class aes:
    
    def __init__(self,s_box):
        self.s_box = s_box

    def key_expension(self, key):
        '''
            todo...
        '''
        key_list = [key]*10
        return key_list
        
    def AddRoundKey(self, input_v,key):
        output = list()
        for a,b in zip(input_v,key):
            output.append(a ^ b)
        return output
    
    def ByteSub(self, input_v):
        for i in range(len(input_v)):
             input_v[i] = self.s_box[input_v[i]]
                
    def ShiftRow(input_v, c=4):
        for i in range(floor(len(input_v)/c)):
            j = i*c
            a = input_v[j:j+c]
            if(i>c):
                raise ValueError('not N x N matrix')
            input_v[j:j+c-i] = a[i:]
            input_v[j+c-i:j+c] = a[:i]

    def x2_gf(r):
        '''
                from https://en.wikipedia.org/wiki/Rijndael_MixColumns
        '''
        a=[]
        b=[]
        for c in range(0,4):
            a[c] = r[c]
            #/* h is 0xff if the high bit of r[c] is set, 0 otherwise */
            h = r[c] >> 7 #/* arithmetic right shift, thus shifting in either zeros or ones */
            b[c] = r[c] << 1 #/* implicitly removes high bit because b[c] is an 8-bit char, so we xor by 0x1b and not 0x11b in the next line */
            b[c] = b[c] ^ 0x1B & h #/* Rijndael's Galois field */
        return a,b
        
    def MixColumn(self, input_v, c=4):
        for i in range(floor(len(input_v)/c)):
            a = list()
            for i in range(floor(len(input_v)/c)):
                a.append(input_v[i+c*j])
            '''
                get Galois Multiplication lookup tables for 2 times
            '''
            a,twotimes = self.x2_gf(a)
            for i in range(floor(len(input_v)/c)):
                input_v[i+c*j] = twotimes[i] ^ a[i+3 & 0x03] ^ a[i+2 & 0x03] ^ twotimes[i+1 & 0x03] ^ a[i+1 & 0x03] & 0xFF
        
    def SBoxInit(self):
        self.s_box = \
            [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67,
            0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59,
            0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7,
            0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1,
            0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05,
            0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83,
            0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29,
            0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b,
            0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa,
            0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c,
            0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc,
            0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec,
            0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19,
            0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee,
            0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49,
            0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
            0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4,
            0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6,
            0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70,
            0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9,
            0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e,
            0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1,
            0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0,
            0x54, 0xbb, 0x16]
            
    def encrypto(self, input_v, key):
        i = 0
        self.SBoxInit()
        keys = self.key_expension(key)
        output = copy(input_v)
        
        for i in range(0,10):
            output = self.AddRoundKey(output, keys[i])
            self.ByteSub(output)
            self.ShiftRow(output)
            self.MixColumn(output)

        self.ByteSub(output)
        self.ShiftRow(output)
        output = self.AddRoundKey(output, keys[i])
        
        return output

In [2]:
# import
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util import Counter
import codecs
import os

## Question 1 and 2

In [3]:
def decrypto(key,cipher,iv = Random.new().read(AES.block_size), mode=AES.MODE_CBC):
    cipher = codecs.decode(cipher, 'hex')
    
    if mode == AES.MODE_CTR:
        ctr = Counter.new(AES.block_size*8,initial_value=int(iv,16))
        decryptor = AES.new(key, mode, counter=ctr)
        plain = decryptor.decrypt(cipher[AES.block_size:])
        return plain
    
    else:
        decryptor = AES.new(key, mode, iv)
        plain = decryptor.decrypt(cipher)
        return plain[AES.block_size:]

In [4]:
CBC_key1 = codecs.decode('140b41b22a29beb4061bda66b6747e14', 'hex')
CBC_Ciphertext1 = '4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81'
iv = codecs.decode(CBC_Ciphertext1, 'hex')[:16]
decrypto(CBC_key1,CBC_Ciphertext1,iv)

b'Basic CBC mode encryption needs padding.\x08\x08\x08\x08\x08\x08\x08\x08'

In [5]:
CBC_key2 = codecs.decode('140b41b22a29beb4061bda66b6747e14', 'hex')
CBC_Ciphertext2 = '5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253'
iv = codecs.decode(CBC_Ciphertext2, 'hex')[:16]
decrypto(CBC_key2,CBC_Ciphertext2,iv)

b'Our implementation uses rand. IV\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'

## Question 3 and 4

In [6]:
CTR_key1 = codecs.decode('36f18357be4dbd77f050515c73fcf9f2', 'hex')
CTR_Ciphertext_1 = '69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329'
iv = codecs.encode(codecs.decode(CTR_Ciphertext_1, 'hex')[:16],'hex')

decrypto(CTR_key1, CTR_Ciphertext_1, iv, mode=AES.MODE_CTR)

b'CTR mode lets you build a stream cipher from a block cipher.'

In [7]:
CTR_Ciphertext_2 = '770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451'
iv = codecs.encode(codecs.decode(CTR_Ciphertext_2, 'hex')[:16],'hex')
decrypto(CTR_key1, CTR_Ciphertext_2, iv, mode=AES.MODE_CTR)

b'Always avoid the two time pad!'